In [12]:
from google.cloud import storage
from google.cloud import kms

from google.oauth2 import service_account

import firebase_admin
from firebase_admin import auth, credentials, firestore

from cryptography.fernet import Fernet

GCP params

In [2]:
# SERVICE_ACCOUNT_FILE = '../firebase-adminsdk.json'
KMS_SERVICE_ACCOUNT_FILE = '../../kms-admin.json'

project_id = 'wallet-login-45c1c'

KMS params

In [3]:
# Build the parent name from the project and location.
parent = f"projects/{project_id}/locations/global"

# Build the key ring name.
key_ring_name = f"{parent}/keyRings/gladius-key-ring"

# Build the key name.
key_name = f"{key_ring_name}/cryptoKeys/gladius-key"

print(key_name)

projects/wallet-login-45c1c/locations/global/keyRings/gladius-key-ring/cryptoKeys/gladius-key


#### Connect to KMS

In [8]:

# Create the credentials object from the service account file.
kms_credentials = service_account.Credentials.from_service_account_file(KMS_SERVICE_ACCOUNT_FILE)

# Create the client with the credentials.
kms_client  = kms.KeyManagementServiceClient(credentials=kms_credentials)

### Load an encryption key from a file in a bucket 

In [13]:


# Create a storage client
storage_client = storage.Client(project='wallet-login-45c1c', 
                                credentials=service_account.Credentials.from_service_account_file('../../firebase-adminsdk.json'))

# Get the bucket and blob (file) objects
bucket_name = 'gladius-backend'
file_name = 'gladius-key-encryption_key-ciphertext.txt'
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_name)

# Download the file contents as bytes
encryption_key = blob.download_as_bytes()



### Initiate Fernet
https://cryptography.io/en/latest/fernet/

In [16]:
f = Fernet(encryption_key)

In [17]:
response = kms_client.encrypt(request={"name": key_name, "plaintext": encryption_key})
encrypted_encryption_key = response.ciphertext

In [18]:
encrypted_encryption_key

b"\n$\x00\x04\xc9Y\x9b\x8c\xe2/@=\xd4|\xe8\x15\x7f\x8c\xc1\x01\xfb\xd5\xba$\xfbhM\xca\x04\xf9\x9ek\x92\xa0?\xb7c\x98\x12U\x00\x06\x16!\xa1\x8a\x9c\xb7\x0bG}w\xe6=\xbf\x92[\x02\ra\xc3-\xcb\x9f_\xc0\xa4\xe3\xd64xD:\xf3\xd3O\x08\x10\xf7W\xfa\xeas\xddX\xbf\xa2\x9cG\x15\x0e3<\xfeZB\xb7'j\xc7\x1f\xef\x8f^\xe1\xe0\x1d`\xa2\xf3\x82{s\xf3\x10\xdd \x83\xf5\x0ej\xa2!\xe0\x0e"

#### Connect to Firebase

In [ ]:
firebase_cred = credentials.Certificate('../../firebase-adminsdk.json')
firebase_admin.initialize_app(firebase_cred)
db = firestore.client()

In [26]:
users_ref = db.collection("users")

# Get all documents from the users collection.
users_docs = users_ref.stream()

for user_doc in users_docs:
    try:
        user_data = user_doc.to_dict()
        user_name = user_data.get("name")
        # print(user_name)  # Consider removing or securing this print in production
        if user_name == 'test03':
            stellar_secret = user_data.get("stellar_secret:", "Field not found")
            encrypted_stellar_secret = f.encrypt(stellar_secret.encode())
            user_doc.reference.update({"encrypted_stellar_secret": encrypted_stellar_secret})
        
            # Store encrypted_encryption_key in user doc
            user_doc.reference.update({"encrypted_encryption_key": encrypted_encryption_key})
                
    except Exception as e:
        print(f"Error processing document {user_doc.id}: {e}")
    